In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## Calls and Functions for reading data downloaded from SEC website 
header = "SETTLEMENT DATE|CUSIP|SYMBOL|QUANTITY (FAILS)|DESCRIPTION|PRICE"

path = r'C:\Users\watso\Desktop\TherinFintech\GME_Shenanigans\SEC_Files_CSV' # USE YOUR PATH 
all_files = glob.glob(path + "/*.csv")

GME_CUSIP_number = "36467W109"   
GME_symbol = 'GME'

CUSIP_number = "36467W109"   # Default  
symbol = 'GME'               # Default

# Main function - passing the CUSIP number (most important) and setting the symbol will deliver
#                 FTD data sorted by CUSIP number 
# All other functions here exist to support this 
def return_dataframe(cusip_number,symbol):
    df = read_ftd_data_any_stock(cusip_number,symbol)
    df = fix_dataframe(df)
    return df

# Use return_dataframe as your function to call FTD Data 


def read_ftd_file(csv_path, cusip_number, symbol):
    data = pd.read_csv(
    Path(csv_path),
    index_col=False
    )
    
    data[header] = data[header].str.replace('|',',')
    new_data = data.rename(columns={header:'SETTLEMENT_DATE,CUSIP,SYMBOL,QUANTITY_FAILS,DESCRIPTION,PRICE'})
    Header = "Header"
    new_data = data.rename(columns={header:Header})
    
    new_data = new_data.Header.str.split(",",expand=True)
    new_data = new_data.rename(columns={0:'Date',1:'CUSIP',2:'SYMBOL',3:'QUANTITY_FAILS',4:'DESCRIPTION',5:'PRICE'})
    
    new_data = new_data.set_index("CUSIP")
    new_data = new_data.loc[cusip_number]
    
    new_data = new_data[['Date','QUANTITY_FAILS','PRICE']]
    new_data.rename(columns={'PRICE':symbol},inplace=True)
    new_data.set_index('Date',inplace=True)  
    
    new_data = new_data.reset_index()
    new_data['Date'] = pd.to_datetime(new_data['Date'])
    new_data.set_index('Date',inplace=True)
    new_data = new_data[['QUANTITY_FAILS']]
    new_data.rename(columns={'QUANTITY_FAILS':symbol+'_QUANTITY_FAILS'},inplace=True)
    
    return new_data

def read_ftd_data_any_stock(cusip_number,symbol):
    new_df = pd.DataFrame()
    for filename in all_files:
        if filename == 0:
            new_df = read_ftd_file(csv_path=filename ,cusip_number=cusip_number,symbol=symbol)
        else:
            new_df2 = read_ftd_file(csv_path=filename ,cusip_number=cusip_number,symbol=symbol)
            new_df = concat_df(new_df,new_df2)
            
    return new_df

def fix_dataframe(dataframe):
    # For whatever reason, exporting data to CSV file, then reimporting back in using Pandas
    # solves all issues when it comes to reading the data in the column 
    dataframe.to_csv('../FilesTemp/temp.csv')
    dataframe = pd.read_csv(
        Path('../FilesTemp/temp.csv'),
        infer_datetime_format=True,
        parse_dates=True,
        index_col="Date",
    )
    dataframe.sort_index(inplace=True)
    dataframe.drop_duplicates(inplace=True)
    return dataframe

def concat_df(df_1,df_2):
    df = pd.concat([df_1,df_2],axis='rows')
    return df

In [3]:
## IEX Setup and Test 

# Don't seem to need to use real API Key 
# Sandbox API key works just fine for data 

#iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

base_url = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

#token = os.environ.get('IEX_API_KEY')
test_token = os.environ.get('IEX_TEST_API_KEY')

test_resp = requests.get(base_url + 'status')
test_resp

<Response [200]>

In [4]:
## IEX Calls and Functions 

def get_chart(stock_ticker, chart_range='14m'):
    resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    return df 

def get_close_price(stock_ticker, chart_range='1y'):

    resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    df = df[['close']]
    #df.rename(columns={'close':stock_ticker},inplace=True)
    
    return df

def get_daily_data(stock_ticker, chart_range='1y'):

    resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    df = df[['close','open','low','high','volume']]
    #df.rename(columns={'close':stock_ticker},inplace=True)
    
    return df

def join_ftd_and_close(ftd_df,close_df):
    merged_df = ftd_df.merge(close_df, how='inner',right_index=True, left_index=True)
    return merged_df

In [5]:
def do_all(cusip_number,symbol,chart_range='1y'):
    # Default chart_range value = '1y' 
    # Get FTD Data
    ftd_df = return_dataframe(cusip_number,symbol)
    # Get Close Data 
    close_df = get_close_price(symbol,chart_range)
    # Merge Data together 
    merged_df = join_ftd_and_close(ftd_df,close_df)
    return merged_df

def do_more(cusip_number,symbol,chart_range='1y'):
    # Default chart_range value = '1y' 
    # Get FTD Data
    ftd_df = return_dataframe(cusip_number,symbol)
    # Get Close Data 
    close_df = get_daily_data(symbol,chart_range)
    # Merge Data together 
    merged_df = join_ftd_and_close(ftd_df,close_df)
    return merged_df

def do_all_and_export(cusip_number,symbol,chart_range='1y'):
    # Default chart_range value = '1y' 
    # Get FTD Data
    ftd_df = return_dataframe(cusip_number,symbol)
    # Get Close Data 
    close_df = get_daily_data(symbol,chart_range)
    # Merge Data together 
    merged_df = join_ftd_and_close(ftd_df,close_df)
    # Export Data 
    merged_df.to_csv('../FilesExport/'+symbol+'_all_data.csv')
    return merged_df

def export_ftd_csv_data(dataframe,symbol):    
    dataframe.to_csv('../FilesExport/'+symbol+'_ftd_data.csv')

In [6]:
gme_ftd_df = return_dataframe(CUSIP_number,symbol)
gme_ftd_df

C:\Users\watso\AppData\Local\Temp/ipykernel_9368/3590283254.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[header] = data[header].str.replace('|',',')


,GME_QUANTITY_FAILS
Date,
2020-08-03,19581
2020-08-04,20822
2020-08-05,9693
2020-08-06,70632
2020-08-10,415
...,...
2021-08-24,7278
2021-08-25,2363
2021-08-26,10983


In [7]:
# gme_chart = get_close_price('GME')
# gme_chart

In [8]:
# combined_df = join_ftd_and_close(gme_ftd_df,gme_chart)
# combined_df

In [9]:
# gme_test = do_all(GME_CUSIP_number,'GME')
# gme_test

In [10]:
# tsla_cusip = "88160R101" 
# tsla_symbol = 'TSLA'

# # tsla_ftd_df = return_dataframe(tsla_cusip,tsla_symbol)
# # tsla_chart = get_close_price(tsla_symbol)
# tsla_ftd_data = do_all(tsla_cusip,tsla_symbol)
# tsla_ftd_data

In [11]:
# aapl_cusip = "037833100" 
# aapl_symbol = 'AAPL'

# # aapl_ftd_df = return_dataframe(aapl_cusip,aapl_symbol)
# # aapl_chart = get_close_price(aapl_symbol)
# aapl_ftd_data = do_all(aapl_cusip,aapl_symbol)
# aapl_ftd_data

In [12]:
%matplotlib inline
def plot_fails(data,symbol):
    plot = data.plot(figsize=(14,5),title=symbol)
    return plot 

In [13]:
#gme_ftd_plot = plot_fails(gme_ftd_df,"GME")

In [14]:
#tsla_ftd_plot = plot_fails(tsla_ftd_df,"TSLA")

In [15]:
#aapl_ftd_plot = plot_fails(aapl_ftd_df,"AAPL")

In [16]:
# def export_csv_data(data,file_title):    
#     data.to_csv('FilesExport/'+file_title+'_ftd_data.csv')
    
# export_csv_data(gme_ftd_df,'GME')

In [17]:
gme_export = do_all_and_export(GME_CUSIP_number,'GME')

C:\Users\watso\AppData\Local\Temp/ipykernel_9368/3590283254.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[header] = data[header].str.replace('|',',')


In [18]:
gme_export

,GME_QUANTITY_FAILS,close,open,low,high,volume
Date,,,,,,
2020-10-02,198726,9.60,9.71,9.50,9.86,4446153
2020-10-05,77344,9.88,9.61,9.61,10.01,2881974
2020-10-06,45728,9.34,9.72,9.10,9.92,4618762
2020-10-07,2112,9.76,9.35,9.35,9.84,3374392
2020-10-08,43355,13.74,9.62,9.25,13.90,78456916
...,...,...,...,...,...,...
2021-08-24,7278,217.65,166.97,166.97,228.00,14847283
2021-08-25,2363,202.73,210.70,193.00,228.00,13334927
2021-08-26,10983,206.51,200.80,200.80,225.00,6528987


In [19]:
tsla_cusip = "88160R101" 
tsla_export = do_all_and_export(tsla_cusip,'TSLA')

C:\Users\watso\AppData\Local\Temp/ipykernel_9368/3590283254.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[header] = data[header].str.replace('|',',')


In [20]:
tsla_export

,TSLA_QUANTITY_FAILS,close,open,low,high,volume
Date,,,,,,
2020-10-02,13296,428.80,423.410,423.4100,447.720,74486843
2020-10-05,6702,428.85,434.020,422.3300,444.860,45257693
2020-10-07,2753,429.60,437.470,423.5320,445.800,43294714
2020-10-08,2318,427.72,441.120,427.3000,447.000,42025903
2020-10-09,4407,437.00,439.840,426.9752,447.054,29972910
...,...,...,...,...,...,...
2021-08-25,3833,725.50,718.650,718.6500,727.790,13225117
2021-08-26,5361,706.95,709.320,706.9500,747.500,13638416
2021-08-27,1949,716.40,736.000,716.4000,748.000,14253354
